<a href="https://colab.research.google.com/github/fender8185/Stock-predict/blob/main/lstm2stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install FinMind==1.5.4

In [ ]:
import tensorflow as tf
import numpy as np
import FinMind
from FinMind.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from datetime import date


In [7]:
params = {
        'stock_id': '0050',
        'start_date': '2018-01-01',
        'end_date': date.today()

    }

data_loader = DataLoader()
stock_data = data_loader.taiwan_stock_daily(**params)
stock_data = data_loader.feature.add_kline_institutional_investors(
    stock_data
)
stock_data = data_loader.feature.add_kline_margin_purchase_short_sale(
    stock_data
)
selected_columns = ['Foreign_Investor_diff','Margin_Purchase_diff','Investment_Trust_diff', 'close' ]
filtered_data = stock_data[selected_columns]
numpy_array = filtered_data.values
all_dates = stock_data['date'].values[1:]

In [ ]:
price_diff = numpy_array[1:, -1] - numpy_array[:-1, -1]
y = price_diff
X = numpy_array[:-1]

split_index = int(len(X) * 0.8)
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = all_dates[:split_index]
test_dates = all_dates[split_index:]

# 數據歸一化
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

# 重塑數據以供LSTM使用
timesteps = 10
X_train_reshaped = []
y_train_reshaped = []
for i in range(len(X_train_scaled) - timesteps):
    X_train_reshaped.append(X_train_scaled[i:i+timesteps])
    y_train_reshaped.append(y_train_scaled[i+timesteps])
X_train_reshaped = np.array(X_train_reshaped)
y_train_reshaped = np.array(y_train_reshaped)

X_test_reshaped = []
y_test_reshaped = []
for i in range(len(X_test_scaled) - timesteps):
    X_test_reshaped.append(X_test_scaled[i:i+timesteps])
    y_test_reshaped.append(y_test_scaled[i+timesteps])
X_test_reshaped = np.array(X_test_reshaped)
y_test_reshaped = np.array(y_test_reshaped)

# LSTM模型
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.fit(X_train_reshaped, y_train_reshaped, epochs=800, batch_size=32)

# 預測
y_pred_train_scaled = model.predict(X_train_reshaped)
y_pred_test_scaled = model.predict(X_test_reshaped)

# 反縮放預測值回原始尺度
y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled)
y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled)


In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 調整日期以匹配預測值
adjusted_train_dates = train_dates[timesteps:]
adjusted_test_dates = test_dates[timesteps:]

fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data', 'Testing Data with Predictions'))

# 添加訓練數據到第一個子圖
fig.add_trace(go.Scatter(x=adjusted_train_dates, y=y_train[timesteps:], mode='lines', name='Training Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=adjusted_train_dates, y=y_pred_train.flatten(), mode='lines', name='Predicted Training Data', marker=dict(symbol='circle-open')), row=1, col=1)

# 添加測試數據的真實值到第二個子圖
fig.add_trace(go.Scatter(x=adjusted_test_dates, y=y_test[timesteps:], mode='lines', name='Real Test Values'), row=1, col=2)
fig.add_trace(go.Scatter(x=adjusted_test_dates, y=y_pred_test.flatten(), mode='lines', name='Predicted Test Values', marker=dict(symbol='circle-open')), row=1, col=2)

fig.update_layout(hovermode='x unified')

# 顯示圖形
fig.show()
